In [24]:
import pandas as pd
import os
import sys
from sklearn import preprocessing
import numpy as np
import random

In [25]:
DATA_DIR = '/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Data/'
PREPROCESS_DIR = 'part1'
PREPROCESS_DIR1 = 'part2'

In [26]:
file_names = []
data_file_names = []

for files in os.listdir(DATA_DIR + PREPROCESS_DIR):
    for files1 in os.listdir(DATA_DIR + PREPROCESS_DIR + '/' + files):
        if 'SecondarySchool' in files1:
            data_file_names.append(DATA_DIR + PREPROCESS_DIR + '/' + files + '/' + files1)
                       
for files in os.listdir(DATA_DIR + PREPROCESS_DIR1):   
    for files1 in os.listdir(DATA_DIR + PREPROCESS_DIR1 + '/' + files):
        if 'SecondarySchool' in files1:
            data_file_names.append(DATA_DIR + PREPROCESS_DIR1 + '/' + files + '/' + files1)
            
print(len(data_file_names))

282


In [27]:
total_electricity_usage_file_data = pd.DataFrame()

for total_electricity_usage_file in data_file_names:
    total_electricity_usage_file_data = total_electricity_usage_file_data.append(pd.read_csv(total_electricity_usage_file), ignore_index=True)

# Drop the Data/time columns, and columns with gas
total_electricity_usage_file_data = total_electricity_usage_file_data.drop(['Date/Time'], axis=1)

# Describe dataframe.
print(total_electricity_usage_file_data.columns.values)
# No. of data points in the dataframe.
print("No. of records in data frame: ", len(total_electricity_usage_file_data))

['Electricity:Facility [kW](Hourly)' 'Fans:Electricity [kW](Hourly)'
 'Cooling:Electricity [kW](Hourly)' 'Heating:Electricity [kW](Hourly)'
 'InteriorLights:Electricity [kW](Hourly)'
 'InteriorEquipment:Electricity [kW](Hourly)' 'Gas:Facility [kW](Hourly)'
 'Heating:Gas [kW](Hourly)' 'InteriorEquipment:Gas [kW](Hourly)'
 'Water Heater:WaterSystems:Gas [kW](Hourly)']
No. of records in data frame:  2470320


In [28]:
# Create a numpy array of dataset from the dataframe
total_electricity_usage_file_data = total_electricity_usage_file_data.values
total_electricity_usage_file_data_normalized = np.zeros(shape=total_electricity_usage_file_data.shape)

for temp in range(total_electricity_usage_file_data.shape[-1]):
    if np.max(total_electricity_usage_file_data[:, temp]) != 0.0:
        total_electricity_usage_file_data_normalized[:, temp] = total_electricity_usage_file_data[:, temp] / np.max(total_electricity_usage_file_data[:, temp])
       
    
print("Shape of data: {}".format(total_electricity_usage_file_data_normalized.shape))

Shape of data: (2470320, 10)


In [29]:
x = total_electricity_usage_file_data_normalized[:, [1,2,3,4,5,6,8,9]]
y = total_electricity_usage_file_data_normalized[:, [0,7]]

print("X shape: {}, Y shape: {}".format(x.shape, y.shape))

# Split data into train, validation, and test data
indices = [temp for temp in range(len(x))]

# Create a random seed
np.random.seed(100)

# randomize samples using shuffle function
random.shuffle(indices)

print("********************")
print("Data information:")
print("********************")
train_indices = [0, int(len(indices) * 0.6)]
val_indices = [int(len(indices) * 0.6),  int(len(indices) * 0.8)]
test_indices = int(len(indices) * 0.8)

print("Train indices: {} \nVal indices: {} \nTest indices: {}".format(train_indices, val_indices, test_indices))

x_train, y_train = x[train_indices[0]: train_indices[1], :], y[train_indices[0]: train_indices[1]]
x_val, y_val = x[val_indices[0]: val_indices[1], :], y[val_indices[0]: val_indices[1]]
x_test, y_test = x[test_indices:, :], y[test_indices:]

# Add an extra dimension
x_train = np.expand_dims(x_train, -1)
x_val = np.expand_dims(x_val, -1)
x_test = np.expand_dims(x_test, -1)

print("X train shape: {}\nY_train shape: {}\nX_val shape: {}\nY_val shape: {}\
      \nX test shape: {}\nY test shape: {}".format(x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape))

# Save data
np.save('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/x_train.npy', x_train)
np.save('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/y_train.npy', y_train)
np.save('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/x_val.npy', x_val)
np.save('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/y_val.npy', y_val)
np.save('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/x_test.npy', x_test)
np.save('/home/phantom/Documents/Electricity_Usage_Forecast_HackUTD/Preprocessed_data/y_test.npy', y_test)

X shape: (2470320, 8), Y shape: (2470320, 2)
********************
Data information:
********************
Train indices: [0, 1482192] 
Val indices: [1482192, 1976256] 
Test indices: 1976256
X train shape: (1482192, 8, 1)
Y_train shape: (1482192, 2)
X_val shape: (494064, 8, 1)
Y_val shape: (494064, 2)      
X test shape: (494064, 8, 1)
Y test shape: (494064, 2)
